## MM BEiT-BERT-BERT

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from torch import optim, nn
from torchvision import models, transforms
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
import pandas as pd

In [ ]:
from transformers import AdamW
import os, sys
sys.path.append('/workout/early-stopping-pytorch')
from pytorchtools import EarlyStopping
from tqdm import tqdm,trange
import time

In [ ]:
from transformers import BeitModel, AutoFeatureExtractor
from transformers import BertTokenizer
from transformers import VisualBertForPreTraining
from transformers import VisionEncoderDecoderModel
from transformers.modeling_outputs import BaseModelOutput
BERTtokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/beit-base-patch16-224-pt22k")

In [ ]:
from prettytable import PrettyTable

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class MemeMQACorpus(torch.utils.data.Dataset):
    """Uses jsonl data to preprocess and serve 
    dictionary of multimodal tensors for model input.
    """

    def __init__(
        self,
        data_path,
        img_dir,
        mode=None,
        balance=False,
        dev_limit=None,
        random_state=0,
    ):

        self.samples_frame = pd.read_json(
            data_path
        )
        
        self.samples_frame = self.samples_frame[self.samples_frame["meme_image"].notnull()]
        self.samples_frame = self.samples_frame[self.samples_frame["ocr"].notnull()]
        self.samples_frame = self.samples_frame[self.samples_frame["entity"].notnull()]
        self.samples_frame = self.samples_frame[self.samples_frame["explanation"].notnull()]
        if mode == "test":
            self.samples_frame = self.samples_frame[self.samples_frame["explanation1"].notnull()]

        self.samples_frame = self.samples_frame.reset_index(
            drop=True
        )
        self.samples_frame.image = self.samples_frame.apply(
            lambda row: (img_dir + '/' + row.meme_image), axis=1
        )
        self.image_transform = Resize((256,256))

    def __len__(self):
        """This method is called when you do len(instance) 
        for an instance of this class.
        """
        return len(self.samples_frame)

    def __getitem__(self, idx):
        """This method is called when you do instance[key] 
        for an instance of this class.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.samples_frame.loc[idx, "meme_image"]  
    
#         ***Get Beit input data***
        file_name = data_dir + self.samples_frame.loc[idx, "meme_image"]
        beit_image_data = Image.open(file_name)
        if beit_image_data.mode != 'RGB':
            beit_image_data = beit_image_data.convert('RGB')            
        beit_inputs = feature_extractor(images=beit_image_data, return_tensors="pt", padding=True)
        beit_inputs['pixel_values'] = beit_inputs['pixel_values'].squeeze()
   
        bert_inputs = self.samples_frame.loc[idx, "question"]  + "\n Options: " + self.samples_frame.loc[idx, "optC"] + "\nContext: " + self.samples_frame.loc[idx, "ocr"]
        
        decoder_text = "Answer: " + self.samples_frame.loc[idx, "entity"] + " BECAUSE " + self.samples_frame.loc[idx, "explanation"]   
            
        sample = {
                "img_name": img_name,
                "bert_inputs": bert_inputs,
                "beit_inputs": beit_inputs,
                "decoder_text": decoder_text
            }
        try:
            sample["decoder_text1"] = "Answer: " + self.samples_frame.loc[idx, "entity"] + " BECAUSE " + self.samples_frame.loc[idx, "explanation1"]
        except:
            pass
        return sample

In [ ]:
BS = 4
train_path = "ANONYMISED"
dev_path = "ANONYMISED"
data_dir = "ANONYMISED"
hm_dataset_train = MemeMQACorpus(train_path, data_dir)
dataloader_train = DataLoader(hm_dataset_train, batch_size=BS,
                        shuffle=True, num_workers=0)
hm_dataset_val = MemeMQACorpus(dev_path, data_dir)
dataloader_val = DataLoader(hm_dataset_val, batch_size=BS,
                        shuffle=True, num_workers=0)

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)
data_time = AverageMeter('Data', ':6.3f')

In [ ]:
from transformers import VisionEncoderDecoderModel
from transformers import BertTokenizer, BertModel
from pathlib import Path

class MM(nn.Module):
    def __init__(self):
        super(MM, self).__init__()        
        self.model_bert = BertModel.from_pretrained("bert-base-uncased")
        self.model_encdec = VisionEncoderDecoderModel.from_encoder_decoder_pretrained("microsoft/beit-base-patch16-224-pt22k", "bert-base-uncased")
        self.model_encdec.config.decoder_start_token_id = BERTtokenizer.cls_token_id
        self.model_encdec.config.pad_token_id = BERTtokenizer.pad_token_id
        
    def forward(self, pixel_values, bert_input_ids, bert_attention_mask, bert_token_type_ids, dec_labels):
        bert_encoder_outputs = self.model_bert(input_ids = bert_input_ids, attention_mask = bert_attention_mask, token_type_ids = bert_token_type_ids, output_hidden_states=True, return_dict=True)
        enc_dec_out = self.model_encdec(pixel_values=pixel_values, encoder_outputs=bert_encoder_outputs, labels=dec_labels, output_hidden_states=True, return_dict=True)
        return enc_dec_out      

try:
    del model
except:
    pass

model = MM()
model.to(device)

In [ ]:
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
count_parameters(model)

In [ ]:
# Train

def train_model(model, patience, n_epochs):
    epochs = n_epochs
    clip = 5
    
    train_acc_list=[]
    val_acc_list=[]
    train_loss_list=[]
    val_loss_list=[]
    train_vencdec_loss_list=[]
    val_vencdec_loss_list=[]
    train_main_loss_list=[]
    val_main_loss_list=[]
    
    # initialize the experiment path
    Path(exp_path).mkdir(parents=True, exist_ok=True)
    # initialize early_stopping object
    chk_file = os.path.join(exp_path, 'checkpoint_'+exp_name+'.pt')
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=chk_file)

    model.train()
    for i in range(epochs):
        print(f"******************************EPOCH - {i}****************************************")
        total_acc_train = 0
        total_loss_train = 0
        total_vencdec_loss_train = 0
        total_main_loss_train = 0
        total_train = 0
        correct_train = 0

        for data in tqdm(dataloader_train, total = len(dataloader_train), desc = "Mini-batch progress"):
            # print(f'------------------Mini Batch - {mbcnt+1}------------------')
            pixel_values_start = time.time()
            try:
                pixel_values = data['beit_inputs']['pixel_values'].to(device)
            except:
                print(data)
                break
            data_time.update(time.time() - pixel_values_start)
            if code_prof:
                print(f"pixel_values processing time: {data_time.val}")
            
            data_time.reset()
            BERT_start = time.time()
            BERTtokens = BERTtokenizer(data['bert_inputs'], padding='max_length', truncation=True, max_length=50)

            input_ids = torch.tensor(BERTtokens["input_ids"]).to(device)
            attention_mask = torch.tensor(BERTtokens["attention_mask"]).to(device)
            token_type_ids = torch.tensor(BERTtokens["token_type_ids"]).to(device)
            data_time.update(time.time() - BERT_start)
            if code_prof:
                print(f"BERT processing time: {data_time.val}")            
            
            
            data_time.reset()
            decoder_labels_start = time.time()
            decoder_labels = BERTtokenizer(data['decoder_text'], padding=True, return_tensors="pt").to(device).input_ids
            data_time.update(time.time() - decoder_labels_start)
            if code_prof:
                print(f"decoder_labels processing time: {data_time.val}")
            
            label = data['label'].to(device)
            model.zero_grad()
            data_time.reset()
            model_start = time.time()
            vencdec_out = model(pixel_values, input_ids, attention_mask, token_type_ids, decoder_labels)
            data_time.update(time.time() - model_start)
            if code_prof:
                print(f"model processing time: {data_time.val}")
            
            vencdec_loss = vencdec_out.loss
            loss = vencdec_loss
            loss.backward()
            optimizer.step()            

            with torch.no_grad():
                total_train += label.size(0)
                total_vencdec_loss_train += vencdec_loss.item()
                total_loss_train += loss.item()
        
        train_acc = 0
        train_loss = 0
        train_vencdec_loss = total_vencdec_loss_train/total_train
        train_main_loss = 0


        # evaluation on validation data phase
        model.eval()
        total_acc_val = 0
        total_loss_val = 0
        total_vencdec_loss_val = 0
        total_main_loss_val = 0
        total_val = 0
        correct_val = 0

        with torch.no_grad():
            for data in dataloader_val:                
                pixel_values_start = time.time()
                pixel_values = data['beit_inputs']['pixel_values'].to(device)
                data_time.update(time.time() - pixel_values_start)
                if code_prof:
                    print(f"pixel_values processing time: {data_time.val}")

                data_time.reset()
                BERT_start = time.time()
                BERTtokens = BERTtokenizer(data['bert_inputs'], padding='max_length', truncation=True, max_length=50)

                input_ids = torch.tensor(BERTtokens["input_ids"]).to(device)
                attention_mask = torch.tensor(BERTtokens["attention_mask"]).to(device)
                token_type_ids = torch.tensor(BERTtokens["token_type_ids"]).to(device)
                data_time.update(time.time() - BERT_start)
                if code_prof:
                    print(f"BERT processing time: {data_time.val}")            


                data_time.reset()
                decoder_labels_start = time.time()
                decoder_labels = BERTtokenizer(data['decoder_text'], padding=True, return_tensors="pt").to(device).input_ids
                data_time.update(time.time() - decoder_labels_start)
                if code_prof:
                    print(f"decoder_labels processing time: {data_time.val}")
                
                label_val = data['label'].to(device)
                model.zero_grad()
                data_time.reset()
                model_start = time.time()
                vencdec_out_val = model(pixel_values, input_ids, attention_mask, token_type_ids, decoder_labels)
                data_time.update(time.time() - model_start)
                if code_prof:
                    print(f"model processing time: {data_time.val}")
                
                vencdec_loss_val = vencdec_out_val.loss
                main_loss_val = 0
                loss_val = vencdec_loss_val
                total_val += label_val.size(0)
                total_vencdec_loss_val += vencdec_loss_val.item()
                total_main_loss_val = 0
                total_loss_val += loss_val.item()
                
        print("Saving model...") 
        torch.save(model.state_dict(), os.path.join(exp_path, "final.pt"))

        val_acc = 0
        val_loss = 0
        val_vencdec_loss = total_vencdec_loss_val/total_val
        val_main_loss = 0
        
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        train_vencdec_loss_list.append(train_vencdec_loss)
        val_vencdec_loss_list.append(total_vencdec_loss_val)
        train_main_loss_list.append(train_main_loss)
        val_main_loss_list.append(total_main_loss_val)
                    
        print(f'Epoch {i+1}: train_acc: {train_acc:.4f} | val_acc: {val_acc:.4f} | train_loss: {train_loss:.4f} | val_loss: {val_loss:.4f} | train_vencdec_loss: {train_vencdec_loss:.4f} | val_vencdec_loss: {val_vencdec_loss:.4f} | train_main_loss: {train_main_loss:.4f} | val_main_loss: {val_main_loss:.4f}')
        with open(os.path.join(exp_path, exp_name+'_base_exp_results.txt'), 'a+') as of:
            of.write(f'Epoch {i+1}: train_acc: {train_acc:.4f} | val_acc: {val_acc:.4f} | train_loss: {train_loss:.4f} | val_loss: {val_loss:.4f} | train_vencdec_loss: {train_vencdec_loss:.4f} | val_vencdec_loss: {val_vencdec_loss:.4f} | train_main_loss: {train_main_loss:.4f} | val_main_loss: {val_main_loss:.4f}\n')
        
        model.train()
        torch.cuda.empty_cache()
        
    return  model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, train_vencdec_loss_list, val_vencdec_loss_list, train_main_loss_list, val_main_loss_list, i


In [ ]:
code_prof = False
exp_name = "MM_BEiT_BERT_BERT"
exp_path = "ANONYMISED"

lr=5e-5
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
n_epochs = 10
# early stopping patience; how long to wait after last time validation loss improved.
patience = 10

model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, train_vencdec_loss_list, val_vencdec_loss_list, train_main_loss_list, val_main_loss_list, i = train_model(model, patience, n_epochs)

In [ ]:
def test_model(model):
    model.eval()
    code_prof = False
    total_loss_test = 0
    total_vencdec_loss_test = 0
    total_main_loss_test = 0
    total_deBERTa_loss_test = 0
    total_test = 0
    correct_test = 0
    generated_result = []
    exp1 = []
    exp2 = []
    ques = []

    with torch.no_grad():
        for _, data in enumerate(dataloader_test, 0):               
            pixel_values_start = time.time()
            pixel_values_test = data['beit_inputs']['pixel_values'].to(device)
            data_time.update(time.time() - pixel_values_start)
            if code_prof:
                print(f"pixel_values processing time: {data_time.val}")

            BERTtokens = BERTtokenizer(data['bert_inputs'], padding='max_length', truncation=True, max_length=50)

            input_ids = torch.tensor(BERTtokens["input_ids"]).to(device)
            attention_mask = torch.tensor(BERTtokens["attention_mask"]).to(device)
            token_type_ids = torch.tensor(BERTtokens["token_type_ids"]).to(device)
            data_time.reset()
            decoder_labels_start = time.time()
            data_time.update(time.time() - decoder_labels_start)
            if code_prof:
                print(f"decoder_labels processing time: {data_time.val}")

            data_time.reset()
            model_start = time.time()            
            bert_encoder_outputs_test = model.model_bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            
            generated_ids = model.model_encdec.generate(pixel_values=pixel_values_test, encoder_outputs=bert_encoder_outputs_test, max_length=150, num_beams = 2, repetition_penalty = 2.5, length_penalty = 1.0)
            preds = [BERTtokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            print(preds)
            generated_result.extend(preds)
            exp1.extend(data["decoder_text"])
            exp2.extend(data["decoder_text1"])
            ques.extend(data["bert_inputs"])
    return generated_result, exp1, exp2, ques

In [ ]:
test_path = "ANONYMISED"
hm_dataset_test = MemeMQACorpus(test_path, data_dir, mode = "test")
dataloader_test = DataLoader(hm_dataset_test, batch_size=BS,
                        shuffle=False, num_workers=0)
generated_result, ref1, ref2, ques = test_model(model)

In [ ]:
dict = {"hyp" : generated_result, "ref1" : ref1, "ref2" : ref2, "ques" : ques}
df1 = pd.DataFrame(dict)
df1.to_csv(exp_name +  ".csv")